In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_url = "C:/Users/kamyar.manshaei/Desktop/Kaggle/Google Competition/train.csv"
train = pd.read_csv(train_url)
test_url = "C:/Users/kamyar.manshaei/Desktop/Kaggle/Google Competition/test.csv"
test = pd.read_csv(test_url)

# <font color='red'>Size of the datasets

In [3]:
print("train size:")
print(train.shape)
print("test size:")
print(test.shape)

train size:
(903653, 56)
test size:
(804684, 54)


--------------------------------------------------------------------------------------------------------------------------------

# <font color='red'>Numeric and Categorical columns

In [4]:
numerics = train._get_numeric_data().columns.values.tolist()
categoricals = [col for col in train.columns.values if col not in numerics]

print('numeric columns:')
print(numerics)
print()
print('categorical columns:')
print(categoricals)

numeric columns:
['isMobile', 'visits', 'hits', 'pageviews', 'bounces', 'newVisits', 'transactionRevenue', 'page', 'targetingCriteria', 'date', 'visitId', 'visitNumber', 'visitStartTime']

categorical columns:
['browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation', 'campaign', 'source', 'medium', 'keyword', 'criteriaParameters', 'isTrueDirect', 'referralPath', 'slot', 'gclId', 'adNetworkType', 'isVideoAd', 'adContent', 'campaignCode', 'channelGrouping', 'fullVisitorId', 'sessionId', 'socialEngagementType']


# <font color='red'>Removing useless columns

### <font color='blue'>Columns appeared only in train set

In [5]:
for col in train.columns.values:
    if col not in test.columns.values:
        print(col)
        print("Number of non-NaN values: ", len(train[col][train[col].isna() == False]))
        print()

transactionRevenue
Number of non-NaN values:  11515

campaignCode
Number of non-NaN values:  1



In [6]:
train.drop('campaignCode',axis=1, inplace=True)
categoricals.remove('campaignCode')

### <font color='blue'>Removing columns containing only NaN

In [7]:
for col in train.columns.values:
    if len(train[col][train[col].isna() != False]) == train.shape[0]:
        print(col)

targetingCriteria


In [8]:
train.drop('targetingCriteria',axis=1, inplace=True)
test.drop('targetingCriteria',axis=1, inplace=True)
numerics.remove('targetingCriteria')

### <font color='blue'>Dropping columns with only one value

In [9]:
for col in train.columns.values:
    if len(train[col].unique()) == 1:
        print(col)
        train.drop(col,axis=1, inplace=True)
        test.drop(col,axis=1, inplace=True)
        if col in categoricals:
            categoricals.remove(col)
        if col in numerics:
            numerics.remove(col)

browserVersion
browserSize
operatingSystemVersion
mobileDeviceBranding
mobileDeviceModel
mobileInputSelector
mobileDeviceInfo
mobileDeviceMarketingName
flashVersion
language
screenColors
screenResolution
visits
cityId
latitude
longitude
networkLocation
criteriaParameters
socialEngagementType


--------------------------------------------------------------------------------------------------------------------------------

# <font color='red'>Finding a unique ID

Just to check how many IDs we have:

In [10]:
[i for i in train.columns.values if 'Id' in i]

['gclId', 'fullVisitorId', 'sessionId', 'visitId']

Here we find columns, in which more than 50% of the values are unique:

In [11]:
for col in train.columns.values:
    if (train[col].nunique() / train.shape[0]) > 0.5 :
        print(col)
        print (train[col].nunique() / train.shape[0])
        print()

fullVisitorId
0.8009744891014582

sessionId
0.9990062557198394

visitId
0.9808001522708385

visitStartTime
0.9817474185334415



In [12]:
for col in test.columns.values:
    if (test[col].nunique() / test.shape[0]) > 0.5 :
        print(col)
        print (test[col].nunique() / test.shape[0])
        print()

fullVisitorId
0.8083781956643851

sessionId
0.9989797237176332

visitId
0.9687082134104816

visitStartTime
0.9696526835378857



As we can see, there is no completely unique column. One probable reason could be presence of replicated rows. Therefore we check if there are replicated rows in the dataset:

In [13]:
train[train.duplicated()]

,browser,operatingSystem,isMobile,deviceCategory,hits,pageviews,bounces,newVisits,transactionRevenue,continent,...,adNetworkType,isVideoAd,adContent,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime


No replicated rows in the datset, but must be checked again after imputation of missing values

### <font color='blue'>Creating a unique ID

According to the description, *sessionId* is the unique ID and contains both *fullVistorId* and *visitId*, but as shown above, there are some duplicated values for *sessionId*, so we firstly check how the rows with identical *sessionIds* differ from each other. As there are many duplicated *sessionIds* and it takes a long time to show all of them, here only 20 duplicated pairs are shown (the number can be increased):

In [14]:
duplicated_ids = train["sessionId"][train["sessionId"].duplicated()].tolist()
from IPython.display import display 
j = 0
for i in duplicated_ids:
    dup = train[train["sessionId"] == i]
    columns = []
    for col in dup.columns.values:
        if(len(dup[col].unique()) > 1):
            columns.append(col)
    display(dup[columns])
    j += 1
    if(j == 20):
        break

,hits,pageviews,networkDomain,date,fullVisitorId,visitStartTime
6760,35,32.0,(not set),20170623,7980925080669177483,1498285182
50181,13,13.0,sbcglobal.net,20170624,7980925080669177483,1498287677


,hits,pageviews,bounces,networkDomain,date,visitStartTime
73865,19,12.0,NaN,sbcglobal.net,20170617,1497682925
78892,1,1.0,1.0,(not set),20170616,1497682650


,hits,pageviews,region,metro,city,date,visitStartTime
73726,6,6.0,not available in demo dataset,not available in demo dataset,not available in demo dataset,20170617,1497683125
80071,4,4.0,Colorado,Denver CO,Denver,20170616,1497681738


,hits,pageviews,date,visitStartTime
83399,33,27.0,20161129,1480492063
89617,11,7.0,20161130,1480492882


,hits,pageviews,bounces,date,visitStartTime
87304,1,1.0,1.0,20161129,1480492790
89962,69,35.0,NaN,20161130,1480492842


...

,hits,pageviews,bounces,date,visitStartTime
7250,1,1.0,1.0,20170312,1489305836
147457,2,2.0,NaN,20170311,1489304458


,hits,pageviews,bounces,date,visitStartTime
128693,1,1.0,1.0,20170310,1489218878
147488,2,2.0,NaN,20170311,1489219967


,hits,pageviews,bounces,region,metro,city,date,visitStartTime
7298,1,1.0,1.0,Tokyo,(not set),Minato,20170312,1489305741
147684,4,4.0,NaN,not available in demo dataset,not available in demo dataset,not available in demo dataset,20170311,1489304431


,hits,pageviews,region,metro,city,date,visitStartTime
129329,2,2.0,not available in demo dataset,not available in demo dataset,not available in demo dataset,20170310,1489219054
147826,6,6.0,Gujarat,(not set),Ahmedabad,20170311,1489219235


,hits,pageviews,bounces,date,visitStartTime
29904,2,2.0,NaN,20170621,1498114691
149158,1,1.0,1.0,20170622,1498115001


It seems we can safely remove rows with duplicated ids, since it seems they only differ in time (and also the number of *hits* and *pageViews*, which apparently makes no differece, but must be checked later). The other differences are mostly caused by missing values. This duplication can be caused by changing the date during the session (the dates of these transactions are consecutive), e.g. transactions which have been started before 12:00 am and lasted until the next day. this claim can be checked later by converting the time column to a timestamp:

In [15]:
train.drop(train[train["sessionId"].duplicated()].index, inplace = True)
test.drop(test[test["sessionId"].duplicated()].index, inplace = True)

In [16]:
train.shape

(902755, 35)

As an alternative we can use the sum of *hits* and *pageViews* for duplicated *sessionIds*, instead of using one of them. This approach seems more logical, if hits and views indicate the total number of *hits* and *pageViews* occured during the transaction. Another important thing to be considered is that if the duplicated IDs represent one transaction, everywhere we have missing values only for one of them, we must replace them by the value of the other row.

### <font color='blue'>Removing useless IDs

As we noticed earlier, *sessionId* contain both *visitId* and *fullVisitorId*. Hence we can remove them without losing any information.:

In [17]:
train.drop('visitId',axis=1, inplace=True)
train.drop('fullVisitorId',axis=1, inplace=True)

test.drop('visitId',axis=1, inplace=True)
test.drop('fullVisitorId',axis=1, inplace=True)

numerics.remove('visitId')
categoricals.remove('fullVisitorId')

--------------------------------------------------------------------------------------------------------------------------------

# <font color='red'> Writing the final dataframe to .csv

In [18]:
train.to_csv('train1.csv', index=False, encoding='utf-8')
test.to_csv('test1.csv', index=False, encoding='utf-8')